In [1]:
import pandas as pd

import os
import lightgbm as lgb
import xgboost as xgb
import numpy as np
import matplotlib.pyplot as plt
import time

from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import Lasso
from sklearn.ensemble import GradientBoostingRegressor

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import ModelCheckpoint


Using TensorFlow backend.
C:\Users\dl\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\dl\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\dl\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\dl\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: Future

In [2]:
df = pd.read_csv('./onion_for_predict.csv')
print(df.shape)
df = df.drop(['거래월'], axis=1)
y = df['가격']
del df['가격']

df

(240, 5)


,년도,월,거래량
0,2000,1,4088280.0
1,2000,2,3557610.0
2,2000,3,4605658.0
3,2000,4,11877910.0
4,2000,5,20421827.0
...,...,...,...
235,2019,8,11930394.0
236,2019,9,12329636.0
237,2019,10,13598455.0
238,2019,11,12481686.0


In [3]:
rand_num = 0
x_train, x_test,y_train, y_test = train_test_split(df, y, test_size=0.3, random_state=rand_num)

gboost = GradientBoostingRegressor(random_state=rand_num)
xgboost = xgb.XGBRegressor(random_state=rand_num)
lightgbm = lgb.LGBMRegressor(random_state=rand_num)
lasso = Lasso(alpha=1.0, random_state=rand_num)
svm = svm.SVC(kernel='rbf', C=1, gamma=0.1, random_state=rand_num)

models = [{'model':xgboost, 'name':'XGBoost'},
          {'model':gboost, 'name': 'GradientBoosting'},
          {'model':lightgbm, 'name':'LightGBM'},
          {'model':lasso, 'name': 'Lasso'}]
result = []
rmse = []
result.append(y_test)

for m in models:
    m['model'].fit(x_train.values, y_train)
    predictions = m['model'].predict(x_test.values) 
    result.append(predictions)
    rmse.append({m['name']:sqrt(mean_squared_error(y_test, predictions))})



In [4]:
model = Sequential()
model.add(Dense(30, input_dim=3, activation='relu'))
model.add(Dense(900, activation='relu'))
model.add(Dense(4096, activation='relu'))
model.add(Dense(1028, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(2, activation='relu'))
model.add(Dense(1))
model.compile(loss='mean_squared_error',
             optimizer='adam',
             metrics=['accuracy'])

# 모델 저장 폴더 설정
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)
    
# 모델 저장 조건 설정
modelpath = MODEL_DIR + '{epoch:02d}-{val_loss:.4f}.hdf5'
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)
history = model.fit(x_train, y_train, validation_split=0.2, epochs=500, batch_size=4, callbacks=[checkpointer])


Train on 134 samples, validate on 34 samples
Epoch 1/500
134/134 [==============================] - 3s 19ms/step - loss: 3133947.9799 - accuracy: 0.0000e+00 - val_loss: 739176.6949 - val_accuracy: 0.0000e+00

Epoch 00001: val_loss improved from inf to 739176.69485, saving model to ./model/01-739176.6949.hdf5
Epoch 2/500
134/134 [==============================] - 2s 17ms/step - loss: 721952.8223 - accuracy: 0.0000e+00 - val_loss: 739148.2721 - val_accuracy: 0.0000e+00

Epoch 00002: val_loss improved from 739176.69485 to 739148.27206, saving model to ./model/02-739148.2721.hdf5
Epoch 3/500
134/134 [==============================] - 2s 17ms/step - loss: 721922.3780 - accuracy: 0.0000e+00 - val_loss: 739114.6140 - val_accuracy: 0.0000e+00

Epoch 00003: val_loss improved from 739148.27206 to 739114.61397, saving model to ./model/03-739114.6140.hdf5
Epoch 4/500
134/134 [==============================] - 2s 17ms/step - loss: 721887.4818 - accuracy: 0.0000e+00 - val_loss: 739078.4118 - val_ac

134/134 [==============================] - 2s 17ms/step - loss: 720682.7435 - accuracy: 0.0000e+00 - val_loss: 737850.0846 - val_accuracy: 0.0000e+00

Epoch 00030: val_loss improved from 737900.31985 to 737850.08456, saving model to ./model/30-737850.0846.hdf5
Epoch 31/500
134/134 [==============================] - 2s 17ms/step - loss: 720633.5765 - accuracy: 0.0000e+00 - val_loss: 737799.5551 - val_accuracy: 0.0000e+00

Epoch 00031: val_loss improved from 737850.08456 to 737799.55515, saving model to ./model/31-737799.5551.hdf5
Epoch 32/500
134/134 [==============================] - 2s 17ms/step - loss: 720584.1343 - accuracy: 0.0000e+00 - val_loss: 737749.6765 - val_accuracy: 0.0000e+00

Epoch 00032: val_loss improved from 737799.55515 to 737749.67647, saving model to ./model/32-737749.6765.hdf5
Epoch 33/500
134/134 [==============================] - 2s 17ms/step - loss: 720534.6488 - accuracy: 0.0000e+00 - val_loss: 737700.4044 - val_accuracy: 0.0000e+00

Epoch 00033: val_loss impro

Epoch 60/500
134/134 [==============================] - 2s 17ms/step - loss: 719180.2192 - accuracy: 0.0000e+00 - val_loss: 736326.9191 - val_accuracy: 0.0000e+00

Epoch 00060: val_loss improved from 736378.05515 to 736326.91912, saving model to ./model/60-736326.9191.hdf5
Epoch 61/500
134/134 [==============================] - 2s 17ms/step - loss: 719129.8456 - accuracy: 0.0000e+00 - val_loss: 736275.2463 - val_accuracy: 0.0000e+00

Epoch 00061: val_loss improved from 736326.91912 to 736275.24632, saving model to ./model/61-736275.2463.hdf5
Epoch 62/500
134/134 [==============================] - 2s 17ms/step - loss: 719078.9021 - accuracy: 0.0000e+00 - val_loss: 736223.8346 - val_accuracy: 0.0000e+00

Epoch 00062: val_loss improved from 736275.24632 to 736223.83456, saving model to ./model/62-736223.8346.hdf5
Epoch 63/500
134/134 [==============================] - 2s 17ms/step - loss: 719028.5410 - accuracy: 0.0000e+00 - val_loss: 736171.6985 - val_accuracy: 0.0000e+00

Epoch 00063: v

Epoch 90/500
134/134 [==============================] - 2s 17ms/step - loss: 717660.5396 - accuracy: 0.0000e+00 - val_loss: 734785.2868 - val_accuracy: 0.0000e+00

Epoch 00090: val_loss improved from 734837.11029 to 734785.28676, saving model to ./model/90-734785.2868.hdf5
Epoch 91/500
134/134 [==============================] - 2s 17ms/step - loss: 717609.6012 - accuracy: 0.0000e+00 - val_loss: 734734.3640 - val_accuracy: 0.0000e+00

Epoch 00091: val_loss improved from 734785.28676 to 734734.36397, saving model to ./model/91-734734.3640.hdf5
Epoch 92/500
134/134 [==============================] - 2s 17ms/step - loss: 717559.1189 - accuracy: 0.0000e+00 - val_loss: 734682.6912 - val_accuracy: 0.0000e+00

Epoch 00092: val_loss improved from 734734.36397 to 734682.69118, saving model to ./model/92-734682.6912.hdf5
Epoch 93/500
134/134 [==============================] - 2s 17ms/step - loss: 717508.3713 - accuracy: 0.0000e+00 - val_loss: 734631.5956 - val_accuracy: 0.0000e+00

Epoch 00093: v

Epoch 120/500
134/134 [==============================] - 2s 17ms/step - loss: 716137.6763 - accuracy: 0.0000e+00 - val_loss: 733241.7426 - val_accuracy: 0.0000e+00

Epoch 00120: val_loss improved from 733293.27574 to 733241.74265, saving model to ./model/120-733241.7426.hdf5
Epoch 121/500
134/134 [==============================] - 2s 17ms/step - loss: 716086.8524 - accuracy: 0.0000e+00 - val_loss: 733191.1985 - val_accuracy: 0.0000e+00

Epoch 00121: val_loss improved from 733241.74265 to 733191.19853, saving model to ./model/121-733191.1985.hdf5
Epoch 122/500
134/134 [==============================] - 2s 17ms/step - loss: 716036.7379 - accuracy: 0.0000e+00 - val_loss: 733139.2794 - val_accuracy: 0.0000e+00

Epoch 00122: val_loss improved from 733191.19853 to 733139.27941, saving model to ./model/122-733139.2794.hdf5
Epoch 123/500
134/134 [==============================] - 2s 17ms/step - loss: 715985.2547 - accuracy: 0.0000e+00 - val_loss: 733088.8162 - val_accuracy: 0.0000e+00

Epoch 0

...

Epoch 360/500
134/134 [==============================] - 2s 17ms/step - loss: 703999.6455 - accuracy: 0.0000e+00 - val_loss: 720937.4779 - val_accuracy: 0.0000e+00

Epoch 00360: val_loss improved from 720988.24265 to 720937.47794, saving model to ./model/360-720937.4779.hdf5
Epoch 361/500
134/134 [==============================] - 2s 17ms/step - loss: 703949.3368 - accuracy: 0.0000e+00 - val_loss: 720886.3713 - val_accuracy: 0.0000e+00

Epoch 00361: val_loss improved from 720937.47794 to 720886.37132, saving model to ./model/361-720886.3713.hdf5
Epoch 362/500
134/134 [==============================] - 2s 17ms/step - loss: 703899.1455 - accuracy: 0.0000e+00 - val_loss: 720834.9081 - val_accuracy: 0.0000e+00

Epoch 00362: val_loss improved from 720886.37132 to 720834.90809, saving model to ./model/362-720834.9081.hdf5
Epoch 363/500
134/134 [==============================] - 2s 17ms/step - loss: 703848.6590 - accuracy: 0.0000e+00 - val_loss: 720783.7537 - val_accuracy: 0.0000e+00

Epoch 0

Epoch 390/500
134/134 [==============================] - 2s 17ms/step - loss: 702491.5261 - accuracy: 0.0000e+00 - val_loss: 719407.8199 - val_accuracy: 0.0000e+00

Epoch 00390: val_loss improved from 719458.66176 to 719407.81985, saving model to ./model/390-719407.8199.hdf5
Epoch 391/500
134/134 [==============================] - 2s 17ms/step - loss: 702441.2687 - accuracy: 0.0000e+00 - val_loss: 719357.4154 - val_accuracy: 0.0000e+00

Epoch 00391: val_loss improved from 719407.81985 to 719357.41544, saving model to ./model/391-719357.4154.hdf5
Epoch 392/500
134/134 [==============================] - 2s 17ms/step - loss: 702391.3316 - accuracy: 0.0000e+00 - val_loss: 719307.0699 - val_accuracy: 0.0000e+00

Epoch 00392: val_loss improved from 719357.41544 to 719307.06985, saving model to ./model/392-719307.0699.hdf5
Epoch 393/500
134/134 [==============================] - 2s 17ms/step - loss: 702340.9389 - accuracy: 0.0000e+00 - val_loss: 719256.4338 - val_accuracy: 0.0000e+00

Epoch 0

Epoch 420/500
134/134 [==============================] - 2s 17ms/step - loss: 700986.6931 - accuracy: 0.0000e+00 - val_loss: 717882.3750 - val_accuracy: 0.0000e+00

Epoch 00420: val_loss improved from 717933.27206 to 717882.37500, saving model to ./model/420-717882.3750.hdf5
Epoch 421/500
134/134 [==============================] - 2s 17ms/step - loss: 700936.3918 - accuracy: 0.0000e+00 - val_loss: 717831.7316 - val_accuracy: 0.0000e+00

Epoch 00421: val_loss improved from 717882.37500 to 717831.73162, saving model to ./model/421-717831.7316.hdf5
Epoch 422/500
134/134 [==============================] - 2s 17ms/step - loss: 700886.6856 - accuracy: 0.0000e+00 - val_loss: 717779.9228 - val_accuracy: 0.0000e+00

Epoch 00422: val_loss improved from 717831.73162 to 717779.92279, saving model to ./model/422-717779.9228.hdf5
Epoch 423/500
134/134 [==============================] - 2s 17ms/step - loss: 700836.1852 - accuracy: 0.0000e+00 - val_loss: 717729.4485 - val_accuracy: 0.0000e+00

Epoch 0

Epoch 450/500
134/134 [==============================] - 2s 17ms/step - loss: 699481.8377 - accuracy: 0.0000e+00 - val_loss: 716356.2757 - val_accuracy: 0.0000e+00

Epoch 00450: val_loss improved from 716407.48897 to 716356.27574, saving model to ./model/450-716356.2757.hdf5
Epoch 451/500
134/134 [==============================] - 2s 17ms/step - loss: 699431.6278 - accuracy: 0.0000e+00 - val_loss: 716304.6434 - val_accuracy: 0.0000e+00

Epoch 00451: val_loss improved from 716356.27574 to 716304.64338, saving model to ./model/451-716304.6434.hdf5
Epoch 452/500
134/134 [==============================] - 2s 17ms/step - loss: 699381.3302 - accuracy: 0.0000e+00 - val_loss: 716253.8346 - val_accuracy: 0.0000e+00

Epoch 00452: val_loss improved from 716304.64338 to 716253.83456, saving model to ./model/452-716253.8346.hdf5
Epoch 453/500
134/134 [==============================] - 2s 17ms/step - loss: 699331.5951 - accuracy: 0.0000e+00 - val_loss: 716202.7426 - val_accuracy: 0.0000e+00

Epoch 0

Epoch 480/500
134/134 [==============================] - 2s 17ms/step - loss: 697977.5075 - accuracy: 0.0000e+00 - val_loss: 714830.8529 - val_accuracy: 0.0000e+00

Epoch 00480: val_loss improved from 714882.12868 to 714830.85294, saving model to ./model/480-714830.8529.hdf5
Epoch 481/500
134/134 [==============================] - 2s 17ms/step - loss: 697927.0970 - accuracy: 0.0000e+00 - val_loss: 714780.7206 - val_accuracy: 0.0000e+00

Epoch 00481: val_loss improved from 714830.85294 to 714780.72059, saving model to ./model/481-714780.7206.hdf5
Epoch 482/500
134/134 [==============================] - 2s 17ms/step - loss: 697876.8330 - accuracy: 0.0000e+00 - val_loss: 714730.0257 - val_accuracy: 0.0000e+00

Epoch 00482: val_loss improved from 714780.72059 to 714730.02574, saving model to ./model/482-714730.0257.hdf5
Epoch 483/500
134/134 [==============================] - 2s 17ms/step - loss: 697827.3757 - accuracy: 0.0000e+00 - val_loss: 714678.1912 - val_accuracy: 0.0000e+00

Epoch 0

In [5]:
modelPred = model.predict(x_test)
Y_prediction = modelPred.flatten()

result.append(Y_prediction)
rmse.append({'keras':sqrt(mean_squared_error(y_test, modelPred))})

dict = {'y': result[0], 'xgboost': result[1], 'gboost': result[2], 'lightgbm':result[3], 'lasso':result[4],'keras': result[5]}

df_res = pd.DataFrame(dict)

print(rmse)
df_res

[{'XGBoost': 212.81114921478058}, {'GradientBoosting': 190.4033675751254}, {'LightGBM': 228.28203229921274}, {'Lasso': 289.5132977544111}, {'SVM': 375.7922926594187}, {'keras': 869.3504623850852}]


,y,xgboost,gboost,lightgbm,lasso,svm,keras
109,1306,717.311707,1055.445950,870.858263,946.482434,658,16.381578
71,495,476.584564,579.801893,607.143671,603.689304,658,16.381578
37,1054,1000.612915,953.254190,893.049535,788.782643,658,16.381578
74,855,667.388367,861.825647,944.360411,810.622214,658,16.381578
108,1040,707.452942,891.770331,847.029707,941.310217,658,16.381578
...,...,...,...,...,...,...,...
218,907,708.074890,767.800326,862.941195,1101.219634,658,16.381578
129,920,889.156128,840.946609,872.465123,803.674870,658,16.381578
73,600,854.888733,793.401041,886.661370,844.322700,658,16.381578
4,616,380.543304,557.662535,440.010000,327.953447,658,16.381578


In [7]:
rand_num = 1
x_train, x_test,y_train, y_test = train_test_split(df, y, test_size=0.3, random_state=rand_num)

gboost = GradientBoostingRegressor(random_state=rand_num)
xgboost = xgb.XGBRegressor(random_state=rand_num)
lightgbm = lgb.LGBMRegressor(random_state=rand_num)
lasso = Lasso(alpha=1.0, random_state=rand_num)
# svm = svm.SVC(kernel='rbf', C=1, gamma=0.1, random_state=rand_num)

models = [{'model':xgboost, 'name':'XGBoost'},
          {'model':gboost, 'name': 'GradientBoosting'},
          {'model':lightgbm, 'name':'LightGBM'},
          {'model':lasso, 'name': 'Lasso'}]
#           {'model':svm, 'name': 'SVM'}]
result = []
rmse = []
result.append(y_test)

for m in models:
    m['model'].fit(x_train.values, y_train)
    predictions = m['model'].predict(x_test.values) 
    result.append(predictions)
    rmse.append({m['name']:sqrt(mean_squared_error(y_test, predictions))})
    
    
# 모델 저장 조건 설정
modelpath = MODEL_DIR + 'r01_{epoch:02d}-{val_loss:.4f}.hdf5'
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)
history = model.fit(x_train, y_train, validation_split=0.2, epochs=500, batch_size=4, callbacks=[checkpointer])


modelPred = model.predict(x_test)
Y_prediction = modelPred.flatten()

result.append(Y_prediction)
rmse.append({'keras':sqrt(mean_squared_error(y_test, modelPred))})

dict = {'y': result[0], 'xgboost': result[1], 'gboost': result[2], 'lightgbm':result[3], 'lasso':result[4],'keras': result[5]}

df_res = pd.DataFrame(dict)

print(rmse)
df_res

Train on 134 samples, validate on 34 samples
Epoch 1/500
134/134 [==============================] - 2s 18ms/step - loss: 777529.3018 - accuracy: 0.0000e+00 - val_loss: 767663.9890 - val_accuracy: 0.0000e+00

Epoch 00001: val_loss improved from inf to 767663.98897, saving model to ./model/r01_01-767663.9890.hdf5
Epoch 2/500
134/134 [==============================] - 2s 17ms/step - loss: 777472.8855 - accuracy: 0.0000e+00 - val_loss: 767608.0901 - val_accuracy: 0.0000e+00

Epoch 00002: val_loss improved from 767663.98897 to 767608.09007, saving model to ./model/r01_02-767608.0901.hdf5
Epoch 3/500
134/134 [==============================] - 2s 17ms/step - loss: 777416.8965 - accuracy: 0.0000e+00 - val_loss: 767552.3438 - val_accuracy: 0.0000e+00

Epoch 00003: val_loss improved from 767608.09007 to 767552.34375, saving model to ./model/r01_03-767552.3438.hdf5
Epoch 4/500
134/134 [==============================] - 2s 17ms/step - loss: 777360.4748 - accuracy: 0.0000e+00 - val_loss: 767496.582

134/134 [==============================] - 2s 18ms/step - loss: 775926.8256 - accuracy: 0.0000e+00 - val_loss: 766067.7224 - val_accuracy: 0.0000e+00

Epoch 00030: val_loss improved from 766121.87500 to 766067.72243, saving model to ./model/r01_30-766067.7224.hdf5
Epoch 31/500
134/134 [==============================] - 2s 18ms/step - loss: 775872.8032 - accuracy: 0.0000e+00 - val_loss: 766013.3217 - val_accuracy: 0.0000e+00

Epoch 00031: val_loss improved from 766067.72243 to 766013.32169, saving model to ./model/r01_31-766013.3217.hdf5
Epoch 32/500
134/134 [==============================] - 2s 18ms/step - loss: 775818.6936 - accuracy: 0.0000e+00 - val_loss: 765959.4136 - val_accuracy: 0.0000e+00

Epoch 00032: val_loss improved from 766013.32169 to 765959.41360, saving model to ./model/r01_32-765959.4136.hdf5
Epoch 33/500
134/134 [==============================] - 2s 17ms/step - loss: 775764.6096 - accuracy: 0.0000e+00 - val_loss: 765905.6949 - val_accuracy: 0.0000e+00

Epoch 00033: va


Epoch 00059: val_loss improved from 764564.84007 to 764513.50184, saving model to ./model/r01_59-764513.5018.hdf5
Epoch 60/500
134/134 [==============================] - 2s 17ms/step - loss: 774312.3918 - accuracy: 0.0000e+00 - val_loss: 764459.2537 - val_accuracy: 0.0000e+00

Epoch 00060: val_loss improved from 764513.50184 to 764459.25368, saving model to ./model/r01_60-764459.2537.hdf5
Epoch 61/500
134/134 [==============================] - 2s 17ms/step - loss: 774258.7446 - accuracy: 0.0000e+00 - val_loss: 764405.7923 - val_accuracy: 0.0000e+00

Epoch 00061: val_loss improved from 764459.25368 to 764405.79228, saving model to ./model/r01_61-764405.7923.hdf5
Epoch 62/500
134/134 [==============================] - 2s 17ms/step - loss: 774205.3857 - accuracy: 0.0000e+00 - val_loss: 764352.7335 - val_accuracy: 0.0000e+00

Epoch 00062: val_loss improved from 764405.79228 to 764352.73346, saving model to ./model/r01_62-764352.7335.hdf5
Epoch 63/500
134/134 [=============================

134/134 [==============================] - 2s 17ms/step - loss: 772764.7687 - accuracy: 0.0000e+00 - val_loss: 762918.1489 - val_accuracy: 0.0000e+00

Epoch 00089: val_loss improved from 762970.51471 to 762918.14890, saving model to ./model/r01_89-762918.1489.hdf5
Epoch 90/500
134/134 [==============================] - 2s 17ms/step - loss: 772711.5858 - accuracy: 0.0000e+00 - val_loss: 762864.7004 - val_accuracy: 0.0000e+00

Epoch 00090: val_loss improved from 762918.14890 to 762864.70037, saving model to ./model/r01_90-762864.7004.hdf5
Epoch 91/500
134/134 [==============================] - 2s 17ms/step - loss: 772658.2491 - accuracy: 0.0000e+00 - val_loss: 762812.2978 - val_accuracy: 0.0000e+00

Epoch 00091: val_loss improved from 762864.70037 to 762812.29779, saving model to ./model/r01_91-762812.2978.hdf5
Epoch 92/500
134/134 [==============================] - 2s 17ms/step - loss: 772605.1567 - accuracy: 0.0000e+00 - val_loss: 762758.9430 - val_accuracy: 0.0000e+00

Epoch 00092: va

134/134 [==============================] - 2s 17ms/step - loss: 771223.3237 - accuracy: 0.0000e+00 - val_loss: 761382.1489 - val_accuracy: 0.0000e+00

Epoch 00118: val_loss improved from 761434.81985 to 761382.14890, saving model to ./model/r01_118-761382.1489.hdf5
Epoch 119/500
134/134 [==============================] - 2s 17ms/step - loss: 771169.6959 - accuracy: 0.0000e+00 - val_loss: 761329.9467 - val_accuracy: 0.0000e+00

Epoch 00119: val_loss improved from 761382.14890 to 761329.94669, saving model to ./model/r01_119-761329.9467.hdf5
Epoch 120/500
134/134 [==============================] - 2s 17ms/step - loss: 771116.8433 - accuracy: 0.0000e+00 - val_loss: 761276.4761 - val_accuracy: 0.0000e+00

Epoch 00120: val_loss improved from 761329.94669 to 761276.47610, saving model to ./model/r01_120-761276.4761.hdf5
Epoch 121/500
134/134 [==============================] - 2s 17ms/step - loss: 771063.6213 - accuracy: 0.0000e+00 - val_loss: 761222.7243 - val_accuracy: 0.0000e+00

Epoch 001

...

134/134 [==============================] - 2s 18ms/step - loss: 755959.0625 - accuracy: 0.0000e+00 - val_loss: 746172.2353 - val_accuracy: 0.0000e+00

Epoch 00408: val_loss improved from 746224.63235 to 746172.23529, saving model to ./model/r01_408-746172.2353.hdf5
Epoch 409/500
134/134 [==============================] - 2s 18ms/step - loss: 755906.3130 - accuracy: 0.0000e+00 - val_loss: 746120.4835 - val_accuracy: 0.0000e+00

Epoch 00409: val_loss improved from 746172.23529 to 746120.48346, saving model to ./model/r01_409-746120.4835.hdf5
Epoch 410/500
134/134 [==============================] - 2s 17ms/step - loss: 755854.3293 - accuracy: 0.0000e+00 - val_loss: 746068.1728 - val_accuracy: 0.0000e+00

Epoch 00410: val_loss improved from 746120.48346 to 746068.17279, saving model to ./model/r01_410-746068.1728.hdf5
Epoch 411/500
134/134 [==============================] - 2s 17ms/step - loss: 755802.0326 - accuracy: 0.0000e+00 - val_loss: 746016.5993 - val_accuracy: 0.0000e+00

Epoch 004

134/134 [==============================] - 2s 17ms/step - loss: 754441.6082 - accuracy: 0.0000e+00 - val_loss: 744661.0074 - val_accuracy: 0.0000e+00

Epoch 00437: val_loss improved from 744713.19118 to 744661.00735, saving model to ./model/r01_437-744661.0074.hdf5
Epoch 438/500
134/134 [==============================] - 2s 18ms/step - loss: 754388.7994 - accuracy: 0.0000e+00 - val_loss: 744609.4835 - val_accuracy: 0.0000e+00

Epoch 00438: val_loss improved from 744661.00735 to 744609.48346, saving model to ./model/r01_438-744609.4835.hdf5
Epoch 439/500
134/134 [==============================] - 2s 17ms/step - loss: 754336.8479 - accuracy: 0.0000e+00 - val_loss: 744555.3952 - val_accuracy: 0.0000e+00

Epoch 00439: val_loss improved from 744609.48346 to 744555.39522, saving model to ./model/r01_439-744555.3952.hdf5
Epoch 440/500
134/134 [==============================] - 2s 17ms/step - loss: 754283.9632 - accuracy: 0.0000e+00 - val_loss: 744503.1544 - val_accuracy: 0.0000e+00

Epoch 004

134/134 [==============================] - 2s 17ms/step - loss: 752925.4128 - accuracy: 0.0000e+00 - val_loss: 743151.0147 - val_accuracy: 0.0000e+00

Epoch 00466: val_loss improved from 743202.76103 to 743151.01471, saving model to ./model/r01_466-743151.0147.hdf5
Epoch 467/500
134/134 [==============================] - 2s 17ms/step - loss: 752872.9715 - accuracy: 0.0000e+00 - val_loss: 743097.5699 - val_accuracy: 0.0000e+00

Epoch 00467: val_loss improved from 743151.01471 to 743097.56985, saving model to ./model/r01_467-743097.5699.hdf5
Epoch 468/500
134/134 [==============================] - 2s 17ms/step - loss: 752820.7509 - accuracy: 0.0000e+00 - val_loss: 743045.4798 - val_accuracy: 0.0000e+00

Epoch 00468: val_loss improved from 743097.56985 to 743045.47978, saving model to ./model/r01_468-743045.4798.hdf5
Epoch 469/500
134/134 [==============================] - 2s 17ms/step - loss: 752768.1427 - accuracy: 0.0000e+00 - val_loss: 742994.1397 - val_accuracy: 0.0000e+00

Epoch 004

134/134 [==============================] - 2s 17ms/step - loss: 751411.3573 - accuracy: 0.0000e+00 - val_loss: 741641.4688 - val_accuracy: 0.0000e+00

Epoch 00495: val_loss improved from 741694.39706 to 741641.46875, saving model to ./model/r01_495-741641.4688.hdf5
Epoch 496/500
134/134 [==============================] - 2s 17ms/step - loss: 751359.2444 - accuracy: 0.0000e+00 - val_loss: 741589.1857 - val_accuracy: 0.0000e+00

Epoch 00496: val_loss improved from 741641.46875 to 741589.18566, saving model to ./model/r01_496-741589.1857.hdf5
Epoch 497/500
134/134 [==============================] - 2s 17ms/step - loss: 751306.6777 - accuracy: 0.0000e+00 - val_loss: 741537.6324 - val_accuracy: 0.0000e+00

Epoch 00497: val_loss improved from 741589.18566 to 741537.63235, saving model to ./model/r01_497-741537.6324.hdf5
Epoch 498/500
134/134 [==============================] - 2s 17ms/step - loss: 751255.0009 - accuracy: 0.0000e+00 - val_loss: 741485.5754 - val_accuracy: 0.0000e+00

Epoch 004

,y,xgboost,gboost,lightgbm,lasso,keras
228,615,646.336731,618.122556,753.184011,1147.905630,32.992378
194,1608,1449.327026,1344.294756,1351.763591,1117.229093,32.992378
88,322,563.555115,621.374462,677.259110,419.935728,32.992378
95,503,637.905151,775.692068,728.368156,719.045512,32.992378
214,1247,1180.448120,1185.523907,1168.846281,1070.902122,32.992378
...,...,...,...,...,...,...
176,526,455.938232,502.732970,565.696169,881.554430,32.992378
167,881,957.577820,972.807861,1004.951918,874.389717,32.992378
150,896,1018.684998,1014.776989,960.751525,924.075032,32.992378
171,636,1034.967041,960.463090,1102.774990,891.101559,32.992378
